In [42]:
import pandas as pd
import time
from multiprocessing import Pool
from sklearn.model_selection import ParameterGrid
import csv
from random import SystemRandom
crypto = SystemRandom()

In [14]:
def preProcessData(stockName, ST, LT):
    df = pd.read_csv("IntradayData_2018/"+stockName+".txt")
    df.columns = ['stock', 'date','time','openingPrice','high','low','closingPrice','volume','e']
    df = df.drop('e', 1)
    for i in ST:
#         df['MA_'+str(i)] = df['openingPrice'].rolling(window=i).mean()
        df['MA_'+str(i)] = df['openingPrice'].ewm(span=i).mean()
    for i in LT:
#         df['MA_'+str(i)] = df['openingPrice'].rolling(window=i).mean()
        df['MA_'+str(i)] = df['openingPrice'].ewm(span=i).mean()
    dateList = list(set(df.date.values))
    dateList = sorted(dateList, key=abs, reverse=False)
    return df, dateList

In [48]:
def convertTime(timeString):
    timeArr = timeString.split(':')
    time = int(timeArr[0])*60 + int(timeArr[1])
    return time

def revConvertTime(timeInt):
    timeHr = timeInt/60
    timeMin = timeInt%60
    return str(timeHr) + ':' + str(timeMin)

def exitTrade(buyPrice, sellPrice, timeBought, timeSold, verbose):
    profit = 100*(sellPrice*(1-0.0002) - buyPrice)/(1.0*buyPrice )
    if(verbose): 
        print(buyPrice, sellPrice, revConvertTime(timeBought), revConvertTime(timeSold), profit)
    return profit

def parseRow(row, ST, LT):
    time = convertTime(row[1].time)
    openingPrice = row[1].openingPrice
    low = row[1].low
    high = row[1].high
    closingPrice = row[1].closingPrice
    listPrice = 0.5*(openingPrice + closingPrice)
    diff = row[1]["MA_"+str(LT)] - row[1]["MA_"+str(ST)]
    return time, openingPrice, low, high, closingPrice, listPrice, diff

def getRandomEntry():
    prob = crypto.random()
    return crypto.random() > prob

In [49]:
def dayScript(df, ST, LT, money, targetPercentage, stopLossPercentage, entryDifference, probability, verbose = False):
    eomDays = 0
    boughtFlag = 0
    dailyTrades = 0
    buyPrice = 0
    sellPrice = 0
    sellMargin = 1 + targetPercentage
    buyMargin = 1 - targetPercentage

    for row in df.iterrows():        
        time, openingPrice, low, high, closingPrice, listPrice, diff = parseRow(row, ST, LT)       
        if(time >= 570 and time <900):
            if(boughtFlag == 0):
                #market enter logic
                if getRandomEntry():
                    if getRandomEntry():
                        #buy
                        buyPrice = listPrice
                        sellPrice = 0
                        timeEntered = time
                        boughtFlag = 1
                        stopLossPrice = buyPrice * (1 - stopLossPercentage)
                        stopLossWindow = buyPrice*stopLossPercentage
                    elif getRandomEntry():
                        #sell
                        sellPrice = listPrice
                        buyPrice = 0
                        timeEntered = time
                        boughtFlag = -1
                        stopGainPrice = sellPrice * (1 + stopLossPercentage)
                        stopGainWindow = sellPrice*stopLossPercentage
            elif(boughtFlag==1):
                #market exit logic (when already bought)
                if(high > buyPrice*sellMargin):
                    percentageProfit = exitTrade(buyPrice, buyPrice*sellMargin, timeEntered, time, verbose)
                    money = money*(1 + percentageProfit/100)
                    dailyTrades+=1
                    boughtFlag = 0
                    buyPrice = 0
#                     print('target exit')
                elif(stopLossPrice > low):
                    percentageProfit = exitTrade(buyPrice, stopLossPrice, timeEntered, time, verbose)
                    money = money*(1 + percentageProfit/100)
                    dailyTrades+=1
                    boughtFlag = 0
                    buyPrice = 0
#                     print('stopLoss exit')
            else:
                #market exit logic (when already sold)
                if(sellPrice*buyMargin > low):
                    percentageProfit = exitTrade(sellPrice*buyMargin, sellPrice, timeEntered, time, verbose)
                    money = money*(1 + percentageProfit/100)
                    dailyTrades+=1
                    boughtFlag = 0
                    sellPrice = 0
#                     print('target exit')
                elif(high > stopGainPrice):
                    percentageProfit = exitTrade(stopGainPrice, sellPrice, timeEntered, time, verbose)
                    money = money*(1 + percentageProfit/100)
                    dailyTrades+=1
                    boughtFlag = 0
                    sellPrice = 0
#                     print('stopLoss exit')
            #trailing stopLoss stopGain calculations.
            if(boughtFlag ==1 ):
                trail = (listPrice - stopLossPrice)
                if(trail > stopLossWindow ):
                    climb = trail - stopLossWindow
                    stopLossPrice+= climb
                    
            elif(boughtFlag ==-1 ):
                trail = (stopGainPrice - listPrice)
                if(trail > stopGainWindow ):
                    climb = trail - stopGainWindow
                    stopGainPrice-= climb
                
        elif(time >= 900):
            if(boughtFlag == 1):
                percentageProfit =exitTrade(buyPrice, closingPrice, timeEntered, time, verbose)
                money = money*(1 + percentageProfit/100)
                dailyTrades+=1
                boughtFlag = 0
                buyPrice = 0
                eomDays += 1
            elif(boughtFlag == -1):
                percentageProfit = exitTrade(closingPrice, sellPrice, timeEntered, time, verbose)
                money = money*(1 + percentageProfit/100)
                dailyTrades+=1
                boughtFlag = 0
                buyPrice = 0
                eomDays += 1

#     print('Total Profit : ',totalProfit)
    return money, dailyTrades, eomDays

In [50]:
def movingAverageExperiment(paramList):
    df, dateList, ST,LT,targetPercentage,stopLossPercentage,entryDifference,probability = paramList
    money = 1
    for date in dateList:
        new_df = df[(df.date == date)]
        updatedMoney, dailyTrades, eomDays = dayScript(new_df, ST, LT,  money, targetPercentage, stopLossPercentage, entryDifference, probability, verbose = False)
#         print(date, money, updatedMoney, dailyTrades)
        money = updatedMoney

#     print('Eom days - ',eomDays)
    yearlyProfitPercentage = (money - 1) * 100
    print(ST,LT,targetPercentage,stopLossPercentage,entryDifference, probability, 'Total Profit %= ', yearlyProfitPercentage)
   
    return yearlyProfitPercentage

In [53]:
def bruteAnalysis(stockName, paramList, pool, threadPoolSize):
    param_grid = {'ST' : ST, 'LT': LT, 'targetPercentage': targetPercentage, 'stopLossPercentage': stopLossPercentage, 'entryDifference':entryDifference, 'probability':probability}
    grid = ParameterGrid(param_grid)

    csvList = []
    df, dateList = preProcessData(stockName, ST, LT)
    i=0
    print('param combination = ' ,len(grid))
    while i<len(grid):
        param_list =[]
        for j in range(threadPoolSize):
            if(i+j < len(grid)):
                params = grid[i+j]
                param_list.append([df, dateList, params['ST'], params['LT'],params['targetPercentage'], params['stopLossPercentage'],params['entryDifference'],params['probability']])
        start = time.time()
        resultList = pool.map(movingAverageExperiment, param_list)
        end = time.time()
        for j in range(len(param_list)):
            params = param_list[j]
            result = resultList[j]
            new_point = {'StockName':stockName, 
                         'ST':params[2], 
                         'LT':params[3], 
                         'targetPercentage':params[4], 
                         'stopLossPercentage':params[5],
                         'entryDifference':params[6], 
                         'probability':params[7],
                         'profitPercentage':result,
                        }
            csvList.append(new_point)
        print('Time - ',end - start)
        i+=threadPoolSize
        print(i,len(grid))
    return csvList

In [58]:
stockList = ['ACC_F1','ASHOKLEY_F1','AXISBANK_F1','BHARTIARTL_F1','RELIANCE_F1','INFY_F1','WIPRO_F1','PNB_F1','SBIN_F1','SUNPHARMA_F1','GRASIM_F1','LUPIN_F1','LT_F1','HINDUNILVR_F1']
# stockList = ['RELIANCE_F1']

In [59]:
# # #old
# ST = [1,4,12,20]
# LT = [100,200,500,1000]
# targetPercentage = [0.01, 0.05]
# stopLossPercentage = [0.002, 0.006, 0.008, 0.010]
# entryDifference = [0.00001, 0.00005, 0.00010, 0.0002]
# # #new
# ST = [4,8,12,16]
# LT = [100,200,300,400,500]
# targetPercentage = [0.01, 0.05, 0.075]
# stopLossPercentage = [0.006, 0.007, 0.008, 0.010]
# entryDifference = [0.00005, 0.000075, 0.0001]
# # # #v3
ST = [4]
LT = [100]
targetPercentage = [0.01]
stopLossPercentage = [0.007, 0.008]
entryDifference = [0.000075]
probability = [0.3]
paramList = [LT, ST, targetPercentage, stopLossPercentage, entryDifference, probability]

In [60]:
# f = csv.writer(open("result.csv", "wb+"))
# f.writerow(["StockName","ST", "LT", "targetPercentage", "stopLossPercentage", "entryDifference","profitPercentage"])

threadPoolSize = 2
pool = Pool(threadPoolSize)
with open('result.csv', 'w') as f_out:
    out_colnames = ["StockName","ST", "LT", "targetPercentage", "stopLossPercentage", "entryDifference","probability","profitPercentage"]
    csv_writer = csv.DictWriter(f_out, fieldnames = out_colnames)
    csv_writer.writeheader()
    for stock in stockList:
        csvList = bruteAnalysis(stock[:-3], paramList,pool,threadPoolSize)
        for point in csvList:
            csv_writer.writerow(point)
pool.terminate()
pool.join()

('param combination = ', 2)
(4, 100, 0.01, 0.007, 7.5e-05, 0.3, 'Total Profit %= ', -36.09050964619706)
(4, 100, 0.01, 0.008, 7.5e-05, 0.3, 'Total Profit %= ', -11.870884722927254)
('Time - ', 19.94083595275879)
(2, 2)
('param combination = ', 2)
(4, 100, 0.01, 0.007, 7.5e-05, 0.3, 'Total Profit %= ', 1.6196557063439476)
(4, 100, 0.01, 0.008, 7.5e-05, 0.3, 'Total Profit %= ', -30.0295572827847)
('Time - ', 23.146183013916016)
(2, 2)
('param combination = ', 2)
(4, 100, 0.01, 0.007, 7.5e-05, 0.3, 'Total Profit %= ', -23.433680735402096)
(4, 100, 0.01, 0.008, 7.5e-05, 0.3, 'Total Profit %= ', -28.939699664576114)
('Time - ', 21.28215789794922)
(2, 2)
('param combination = ', 2)
(4, 100, 0.01, 0.007, 7.5e-05, 0.3, 'Total Profit %= ', -32.81468423620526)
(4, 100, 0.01, 0.008, 7.5e-05, 0.3, 'Total Profit %= ', -41.232889955589215)
('Time - ', 20.505449771881104)
(2, 2)
('param combination = ', 2)
(4, 100, 0.01, 0.007, 7.5e-05, 0.3, 'Total Profit %= ', -28.94544421905695)
(4, 100, 0.01, 0.00